In [2]:
import os

In [3]:
import praw
import openai

In [4]:
client_id = os.getenv("REDDIT_CLIENT")
client_secret = os.getenv("REDDIT_SECRET")

In [7]:
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent="ai sentiment example (by u/karidwyn)"
                    )

In [9]:
for submission in reddit.subreddit("stocks").hot(limit=5):
    print(submission.title)

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
r/Stocks Daily Discussion & Technicals Tuesday - Mar 28, 2023
Alibaba to split into 6 units
Apple (AAPL) to Spend Billions More on Entertainment
Disney will start laying off employees this week, CEO Bob Iger says in memo


In [10]:
subreddit_stocks = reddit.subreddit("stocks")

In [11]:
subreddit_stocks.title

'Stocks - Investing and trading for all'

In [12]:
subreddit_stocks.accounts_active

4881

In [18]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title)
    submission = reddit.submission(post.id)
    #print the top 2 comments per title submission
    counter = 0
    for comment in submission.comments:
        if comment.body == '[deleted]':
            pass
        else:
            print(comment.body)
        counter+=1
        if counter == 2:
            break

Rate My Portfolio - r/Stocks Quarterly Thread March 2023
I am 26, so I have several decades until retirement. My goal is to hold the stocks in my portfolio perpetually, though I'm not reserved to make adjustments and sell when a) I see an opportunity to take profits b) my thesis changes. My long term goal is passive income generation, though I don't chase yields. I don't pay much attention to valuation or macro, as I think that since I plan on holding these stocks for decades while buying every month, those don't matter to me much. I invest regularly every month, so I'm okay with buying at a higher valuation at times and lower valuation at other times. In my opinion, buying at the best possible time is more difficult than buying a good company. That said, if I see an undervalued stock I like, I don't hesitate to buy more and when I make my regular contributions, I try weigh towards stocks that might have been beaten down or seem undervalued. My last review in one of these was [Dec 2021

In [28]:
def get_titles_and_comments(subreddit="stocks",limit=6,num_comments=3,skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    title_and_comments = {}
    
    for counter,post in enumerate(subreddit.hot(limit=limit)):
        
        if counter < skip_first:
            continue
            
        counter += (1-skip_first)
        
        title_and_comments[counter] = ""
        submission = reddit.submission(post.id)
        title = post.title
        
        title_and_comments[counter] += 'Title: '+title+"\n\n"
        title_and_comments[counter] += "Comments: \n\n"
        
        comment_counter = 0
        for comment in submission.comments:
            if not comment.body == "[deleted]":
                title_and_comments[counter] += comment.body+"\n"
                comment_counter += 1
            if comment_counter == num_comments:
                break
    return title_and_comments


In [29]:
titles_and_comments = get_titles_and_comments()

In [30]:
titles_and_comments

{1: 'Title: Apple launches its Pay Later service\n\nComments: \n\nalright, time to finance a pizza\nBig news, especially for AFRM which is -9% on the day.\nI will now be financing every day goods like apples and sandwiches 🤷🏼\u200d♂️ charge it to the game, Apple.\n',
 2: 'Title: Alibaba to split into 6 units\n\nComments: \n\nUnlocks business value, allows it to be carved out and less threatening to the CCP\nWhat does this mean for existing shareholders? Will they get a slice of each of these companies?\nSo that’s why Jack Ma suddenly reappeared\n',
 3: 'Title: Apple (AAPL) to Spend Billions More on Entertainment\n\nComments: \n\nI think content creation is really there to add value to their products. \n\nNetflix has 31.6bn revenue and Apple has 394.3bn revenue.\n\nChasing the competitive streaming dollars is such a poor use of cash. But driving more product sales and bringing people into the Apple ecosystem is far more profitable.\n\nNetflix would rather sell $1000 devices every 2 year

In [31]:
def create_prompt(title_and_comments):

    task = "Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':\n\n"
    return task + title_and_comments

In [32]:
print(titles_and_comments[1])

Title: Apple launches its Pay Later service

Comments: 

alright, time to finance a pizza
Big news, especially for AFRM which is -9% on the day.
I will now be financing every day goods like apples and sandwiches 🤷🏼‍♂️ charge it to the game, Apple.



In [34]:
print(create_prompt(titles_and_comments[1]))

Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':

Title: Apple launches its Pay Later service

Comments: 

alright, time to finance a pizza
Big news, especially for AFRM which is -9% on the day.
I will now be financing every day goods like apples and sandwiches 🤷🏼‍♂️ charge it to the game, Apple.



In [37]:
for key, title_and_comments in titles_and_comments.items():
    
    prompt = create_prompt(title_and_comments)
    
    response = openai.Completion.create(engine="text-davinci-003",
                                                prompt=prompt,
                                                max_tokens=256,
                                                temperature=0,
                                                top_p=1.0,
                                                frequency_penalty=0.0,
                                                presence_penalty=0.0)
    print(title_and_comments)
    print("Sentiment Report from OpenAI: " + response["choices"][0]["text"])
    print("-"*30)

Title: Apple launches its Pay Later service

Comments: 

alright, time to finance a pizza
Big news, especially for AFRM which is -9% on the day.
I will now be financing every day goods like apples and sandwiches 🤷🏼‍♂️ charge it to the game, Apple.

Sentiment Report from OpenAI: 
Company: Apple 
Sentiment: Positive
------------------------------
Title: Alibaba to split into 6 units

Comments: 

Unlocks business value, allows it to be carved out and less threatening to the CCP
What does this mean for existing shareholders? Will they get a slice of each of these companies?
So that’s why Jack Ma suddenly reappeared

Sentiment Report from OpenAI: 
Alibaba
------------------------------
Title: Apple (AAPL) to Spend Billions More on Entertainment

Comments: 

I think content creation is really there to add value to their products. 

Netflix has 31.6bn revenue and Apple has 394.3bn revenue.

Chasing the competitive streaming dollars is such a poor use of cash. But driving more product sales an